# Asset Management & Sustainable Finance
- Sarahnour Ghaith
- Thomas Roiseux

In [ ]:
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt
from scipy.stats import norm

from typing import Dict, Callable

import warnings

warnings.filterwarnings("ignore")

## Exercise 1 - Portfolio optimization and risk budgeting

In [ ]:
corr_mat = np.array(
    [
        [1, 0.5, 0.3, 0.6, 0.4],
        [0.5, 1, 0.3, 0.6, 0.3],
        [0.3, 0.3, 1, 0.6, 0.7],
        [0.6, 0.6, 0.6, 1, 0.3],
        [0.4, 0.3, 0.7, 0.3, 1],
    ],
    dtype=np.float64,
)

r = 0.02  # risk free rate

mu = np.array([0.05, 0.05, 0.06, 0.04, 0.07], dtype=np.float64).T
sigma = np.array([0.2, 0.22, 0.25, 0.18, 0.45], dtype=np.float64).T

### Question 1.a

In [ ]:
cov_mat = np.zeros_like(corr_mat)  # Covariance matrix

for i in range(len(cov_mat)):
    for j in range(i, len(cov_mat)):
        cov_mat[i, j] = corr_mat[i, j] * sigma[i] * sigma[j]
        cov_mat[j, i] = cov_mat[i, j]


print(cov_mat)

### Question 1.b

In [ ]:
sr = (mu - r) / sigma  # Sharpe ratio

for k in range(len(mu)):
    print("Sharpe ratio for asset", k + 1, ":", sr[k])

### Question 2.b

In [ ]:
def solve_qp_problem(gamma: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        gamma (float): gamma parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    x = cp.Variable(len(mu), "x")  # Portfolio weights

    objective = cp.Minimize(
        0.5 * cp.quad_form(x, cov_mat) - gamma * cp.matmul(mu - r, x)
    )
    constraints = [cp.sum(x) == 1, -10 <= x, x <= 10]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return x.value

In [ ]:
optimal_x: Dict[float, np.ndarray] = {}
gammas = [0, 0.1, 0.2, 0.5, 1]

for gamma in gammas:
    sol = solve_qp_problem(gamma)
    print("Optimal portfolio for gamma =", gamma, ":", sol)
    optimal_x[gamma] = sol

In [ ]:
for gamma, sol in optimal_x.items():
    print("Gamma:", gamma)
    expected_return = np.dot(sol.T, mu)
    volatility = sol.T.dot(cov_mat).dot(sol)
    sharpe_ratio = (expected_return - r) / volatility
    print("  Expected return:", expected_return)
    print("  Volatility:", volatility)
    print("  Sharpe ratio:", sharpe_ratio)

### Question 2.c

In [ ]:
precise_gammas = np.linspace(-10, 10, 500)

solutions = [solve_qp_problem(gamma) for gamma in precise_gammas]
returns = np.array([np.dot(sol.T, mu) * 100 for sol in solutions])
volatilities = np.array([sol.T.dot(cov_mat).dot(sol) * 100 for sol in solutions])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    volatilities,
    returns,
    label="Efficient frontier",
)
plt.legend()
plt.grid(visible=True)
plt.xlabel("Volatility (in %)")
plt.ylabel("Expected return (in %)")
plt.title("Efficient frontier")
plt.show()

### Question 2.d

In [ ]:
def bisection_algorithm(
    f: Callable[[float], float],
    a: float,
    b: float,
    target: float,
    /,
    tol: float = 10**-5,
    *,
    max_iter: int = 100,
) -> float:
    """Bisection algorithm

    Args:
        f (function): function to find the root of
        a (float): left bound
        b (float): right bound
        target (float): target value
        tol (float, optional): tolerance. Defaults to 10**-5.
        max_iter (int, optional): maximum number of iterations. Defaults to 100.

    Returns:
        float: root of the function
    """
    if b - a < 0:
        a, b = b, a

    gamma_bar = (a + b) / 2

    if b - a < tol or max_iter == 0:
        return gamma_bar

    f_bar = f(gamma_bar)

    if f_bar < target:
        a = gamma_bar
    else:
        b = gamma_bar

    return bisection_algorithm(f, a, b, target, tol=tol, max_iter=max_iter - 1)

In [ ]:
targets = (16, 20)


def volatilities_function(gamma: float) -> float:
    sol = solve_qp_problem(gamma)
    return sol.T.dot(cov_mat).dot(sol) * 100


for target in targets:
    gamma = bisection_algorithm(
        volatilities_function,
        0,
        100,
        target,
    )
    print("Gamma for target volatility of", target, ":", gamma)
    print("  Expected return:", np.dot(solve_qp_problem(gamma).T, mu) * 100)
    print("  Volatility:", volatilities_function(gamma))
    print(
        "  Sharpe ratio:",
        (np.dot(solve_qp_problem(gamma).T, mu) - r) / volatilities_function(gamma),
    )

### Question 2.e

In [ ]:
sharpe_ratios = (returns - r * 100) / volatilities  # Converting r into %

i = np.argmax(sharpe_ratios)
tp = solve_qp_problem(precise_gammas[i])
print("Optimal portfolio for maximum Sharpe ratio:", tp)
print("Maximum Sharpe ratio:", sharpe_ratios[i])
print("Expected return (in %):", returns[i])
print("Volatility (in %):", volatilities[i])

### Question 2.f
The analytical solutoin can be found by maximizing the Sharpe Ratio. This is exactly what we did in the previous question so we will have the same answer.

### Question 3.a

In [ ]:
def solve_long_only_qp_problem(gamma: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        gamma (float): gamma parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    x = cp.Variable(len(mu), "x")  # Portfolio weights

    objective = cp.Minimize(
        0.5 * cp.quad_form(x, cov_mat) - gamma * cp.matmul(mu - r, x)
    )
    constraints = [cp.sum(x) == 1, 0 <= x, x <= 1]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return x.value

In [ ]:
for gamma in gammas:
    sol = solve_long_only_qp_problem(gamma)
    print("Optimal portfolio for gamma =", gamma, ":", sol)
    print("Gamma:", gamma)
    expected_return = np.dot(sol.T, mu)
    volatility = sol.T.dot(cov_mat).dot(sol)
    sharpe_ratio = (expected_return - r) / volatility
    print("  Expected return (in %):", expected_return * 100)
    print("  Volatility (in %):", volatility * 100)
    print("  Sharpe ratio:", sharpe_ratio)

### Question 3.b

In [ ]:
long_solutions = [solve_long_only_qp_problem(gamma) for gamma in precise_gammas]
long_returns = np.array([np.dot(sol.T, mu) * 100 for sol in long_solutions])
long_volatilities = np.array(
    [sol.T.dot(cov_mat).dot(sol) * 100 for sol in long_solutions]
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    volatilities,
    returns,
    label="Efficient frontier long long/short",
)
plt.plot(
    long_volatilities,
    long_returns,
    label="Efficient frontier long only",
)
plt.legend()
plt.grid(visible=True)
plt.xlabel("Volatility (in %)")
plt.ylabel("Expected return (in %)")
plt.title("Efficient frontier")
plt.show()

### Question 3.c

In [ ]:
for target in targets:
    gamma = bisection_algorithm(
        volatilities_function,
        0,
        100,
        target,
    )
    print("Gamma for target volatility of", target, ":", gamma)
    print("  Expected return:", np.dot(solve_long_only_qp_problem(gamma).T, mu) * 100)
    print("  Volatility:", volatilities_function(gamma))
    print(
        "  Sharpe ratio:",
        (np.dot(solve_qp_problem(gamma).T, mu) - r) / volatilities_function(gamma),
    )

### Question 3.d

In [ ]:
sharpe_ratios = (long_returns - r * 100) / long_volatilities  # Converting r into %

i = np.argmax(sharpe_ratios)
tp = solve_long_only_qp_problem(precise_gammas[i])
print("Optimal portfolio for maximum Sharpe ratio:", tp)
print("Maximum Sharpe ratio:", sharpe_ratios[i])
print("Expected return (in %):", returns[i])
print("Volatility (in %):", volatilities[i])

### Question 3.e

In [ ]:
beta = cov_mat.dot(tp) / tp.T.dot(cov_mat).dot(tp)

for i, value in enumerate(beta):
    print("Beta for asset", i + 1, ":", value)

### Question 4.a-d
Theoretical questions, no code needed.
### Question 4.e

## Equity portfolio optimization with net zero objectives
### Question 1.a

In [115]:
beta = np.array(
    [0.95, 1.05, 0.45, 1.40, 1.15, 0.75, 1.00, 1.20, 1.10, 0.8, 0.7]
).reshape(-1, 1)
sigma_mat = (
    np.diag(
        [0.262, 0.329, 0.211, 0.338, 0.231, 0.259, 0.265, 0.271, 0.301, 0.274, 0.228]
    )
    ** 2
)

weights = np.array(
    [8.20, 12.30, 6.90, 3.10, 13.20, 12.60, 10.20, 23.00, 4.50, 2.80, 3.20]
) # vector of weights of the assets in the benchmark portfolio
weights = weights / 100 # Converting from percentage to decimal

# Computing covariance matrix
sigma_m = 0.2**2

cov_mat: np.ndarray = sigma_m * (beta.dot(beta.T)) + sigma_mat
cov_mat 

array([[0.104744, 0.0399  , 0.0171  , 0.0532  , 0.0437  , 0.0285  ,
        0.038   , 0.0456  , 0.0418  , 0.0304  , 0.0266  ],
       [0.0399  , 0.152341, 0.0189  , 0.0588  , 0.0483  , 0.0315  ,
        0.042   , 0.0504  , 0.0462  , 0.0336  , 0.0294  ],
       [0.0171  , 0.0189  , 0.052621, 0.0252  , 0.0207  , 0.0135  ,
        0.018   , 0.0216  , 0.0198  , 0.0144  , 0.0126  ],
       [0.0532  , 0.0588  , 0.0252  , 0.192644, 0.0644  , 0.042   ,
        0.056   , 0.0672  , 0.0616  , 0.0448  , 0.0392  ],
       [0.0437  , 0.0483  , 0.0207  , 0.0644  , 0.106261, 0.0345  ,
        0.046   , 0.0552  , 0.0506  , 0.0368  , 0.0322  ],
       [0.0285  , 0.0315  , 0.0135  , 0.042   , 0.0345  , 0.089581,
        0.03    , 0.036   , 0.033   , 0.024   , 0.021   ],
       [0.038   , 0.042   , 0.018   , 0.056   , 0.046   , 0.03    ,
        0.110225, 0.048   , 0.044   , 0.032   , 0.028   ],
       [0.0456  , 0.0504  , 0.0216  , 0.0672  , 0.0552  , 0.036   ,
        0.048   , 0.131041, 0.0528  , 0.038

In [113]:
# Computing correlation matrix
corr_mat = np.zeros_like(cov_mat)

for i in range(len(corr_mat)):
    for j in range(i, len(corr_mat)):
        corr_mat[i, j] = cov_mat[i, j] / (np.sqrt(sigma_mat[i, i] * sigma_mat[j, j]))
        corr_mat[j, i] = corr_mat[i, j]

corr_mat

array([[1.52590175, 0.46288777, 0.30932311, 0.60074981, 0.72205149,
        0.41999469, 0.54731384, 0.64223543, 0.53003982, 0.4234691 ,
        0.44529262],
       [0.46288777, 1.40742417, 0.27225976, 0.52876747, 0.63553468,
        0.36967058, 0.48173424, 0.56528225, 0.46653001, 0.37272868,
        0.39193729],
       [0.30932311, 0.27225976, 1.18193661, 0.35334698, 0.42469379,
        0.24703105, 0.3219172 , 0.37774785, 0.31175702, 0.24907462,
        0.2619107 ],
       [0.60074981, 0.52876747, 0.35334698, 1.68625048, 0.82481621,
        0.47976971, 0.62520933, 0.73364047, 0.60547681, 0.48373861,
        0.50866812],
       [0.72205149, 0.63553468, 0.42469379, 0.82481621, 1.99136073,
        0.57664343, 0.75144981, 0.88177505, 0.72773295, 0.58141372,
        0.61137693],
       [0.41999469, 0.36967058, 0.24703105, 0.47976971, 0.57664343,
        1.33541539, 0.43709478, 0.51290088, 0.42329943, 0.33819012,
        0.35561878],
       [0.54731384, 0.48173424, 0.3219172 , 0.62520933, 0.

In [114]:
sector_volatility = np.sqrt(np.diag(cov_mat))
sector_volatility

array([0.32364178, 0.39030885, 0.22939268, 0.43891229, 0.32597699,
       0.29930085, 0.33200151, 0.36199586, 0.37282838, 0.31729482,
       0.26755186])

### Question 1.b

The volatility of the benchmark can be computed as follows:
$$ \sigma_{\text{benchmark}} = \sqrt{b^{\mathsf T} \Sigma b}$$
where $b$ is the vector of the benchmark weights.

In [119]:
sigma_benchmark = np.sqrt((weights.T).dot(cov_mat).dot(weights))
print("The volatility of the benchmark portfolio is: ", sigma_benchmark)

The volatility of the benchmark portfolio is:  0.22211447894047792


### Question 1.c

In [116]:
weighted_average = np.sum(weights * beta)
weighted_average

10.549999999999999

Some assets have a huge ponderation where others have a smaller one.

### Question 1.d

In [120]:
implied_risk_premia: np.ndarray = (
    0.25 * cov_mat.dot(weights) / np.sqrt(weights.T.dot(cov_mat).dot(weights))
)
implied_risk_premia

array([0.04911049, 0.06226276, 0.02371947, 0.06702306, 0.05970823,
       0.04328309, 0.05308856, 0.07304367, 0.05411786, 0.03838711,
       0.03339076])

In [121]:
r_m = 0.25 * 0.2 + 0.03

expected_returns = 0.03 + beta.T * (r_m - 0.03)
expected_returns

array([[0.0775, 0.0825, 0.0525, 0.1   , 0.0875, 0.0675, 0.08  , 0.09  ,
        0.085 , 0.07  , 0.065 ]])

### Question 1.e

In [122]:
sci12 = np.array([24, 54, 47, 434, 19, 21, 105, 23, 559, 89, 1655]) # vector of carbon intensity of the assets

ci_b = np.sum(weights * sci12) # carbon intensity of the benchmark portfolio
ci_b

127.06800000000001

In [123]:
cm12 = np.array([-2.8, -7.2, -1.8, -1.5, -8.3, -7.8, -8.5, -4.3, -7.1, -2.7, -9.9]) # vector of carbon momentum of the assets
cm12 = cm12 / 100 # Converting from percentage to decimal

cm_b = np.sum(weights * cm12) # carbon momentum of the benchmark portfolio
cm_b

-0.059322

In [124]:
gii = np.array([0, 1.5, 0, 0.7, 0, 0, 2.4, 0.2, 0.8, 1.4, 8.4]) # vector of green intensity of the assets
gii = gii / 100 # Converting from percentage to decimal

gi_b = np.sum(weights * gii) # green intensity of the benchmark portfolio
gi_b 

0.00841

### Question 2.b

In [125]:
def solve_esg_qp_problem(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci_b,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [126]:
optimal_pfs12 = {}

for t in (0, 1, 2, 5, 10):

    sol = solve_esg_qp_problem(t)

    print("Optimal portfolio for t =", t, ":", sol)

    tracking_eror_volatility = np.sqrt(
        (sol - weights).T.dot(cov_mat).dot(sol - weights)
    )

    print("Tracking error volatility:", tracking_eror_volatility)

    carbon_intensity = sci12.dot(sol)

    print("Carbon intensity:", carbon_intensity)

    carbon_momentum = cm12.dot(sol)

    print("Carbon momentum:", carbon_momentum)
    green_intensity = gii.dot(sol)

    print("Green intensity:", green_intensity)

    reduction_rate = 1 - (sci12.dot(sol)) / (ci_b)

    print("Reduction rate:", reduction_rate)
    optimal_pfs12[t] = sol

Optimal portfolio for t = 0 : [ 9.52079102e-02  3.33688277e-02  4.46521686e-01 -7.42421262e-22
  2.18675153e-02  1.78335631e-01  6.53289731e-02  3.57770471e-21
 -6.66484141e-22  1.34675829e-01  2.46936267e-02]
Tracking error volatility: 0.13190031530001348
Carbon intensity: 88.9476
Carbon momentum: -0.04046482958791959
Green intensity: 0.006028154022616099
Reduction rate: 0.30000000000000016
Optimal portfolio for t = 1 : [ 9.57588961e-02  3.36596358e-02  4.47503538e-01 -5.57242261e-22
  2.25203947e-02  1.78961739e-01  6.56820507e-02  2.85476259e-21
 -5.07829487e-22  1.35087599e-01  2.08261463e-02]
Tracking error volatility: 0.1319822728328824
Carbon intensity: 82.72126800000001
Carbon momentum: -0.04028014295556798
Green intensity: 0.005721886439325951
Reduction rate: 0.349
Optimal portfolio for t = 2 : [ 9.62713129e-02  3.39300875e-02  4.48416659e-01 -3.85718648e-22
  2.31275725e-02  1.79544020e-01  6.60104129e-02  2.18373323e-21
 -3.60783291e-22  1.35470546e-01  1.72293896e-02]
Track

### Question 2.c

In [130]:
scii13 = np.array([78, 203, 392, 803, 55, 124, 283, 123, 892, 135, 1867]) # vector of carbon intensity of the assets for scope 1, 2 and 3

ci13_b = np.sum(weights * scii13) # carbon intensity of the benchmark portfolio for scope 1, 2 and 3
ci13_b

267.01

In [128]:
cmi13 = np.array([-0.8, -1.6, -0.1, -0.2, -1.9, -2.0, -2.5, 2.1, -3.6, -0.8, -6.8]) # vector of carbon momentum of the assets for scope 1, 2 and 3
cmi13 = cmi13 / 100 # Converting from percentage to decimal

In [132]:
def solve_esg_qp_problem13(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(scii13 * w) <= 0.7 * (0.93**t) * ci13_b,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [131]:
optimal_pfs13 = {}

for t in (0, 1, 2, 5, 10):

    sol = solve_esg_qp_problem13(t)

    print("Optimal portfolio for t =", t, ":", sol)

    tracking_eror_volatility = np.sqrt(
        (sol - weights).T.dot(cov_mat).dot(sol - weights)
    )

    print("Tracking error volatility:", tracking_eror_volatility)

    carbon_intensity = scii13.dot(sol)

    print("Carbon intensity:", carbon_intensity)

    carbon_momentum = cmi13.dot(sol)

    print("Carbon momentum:", carbon_momentum)
    green_intensity = gii.dot(sol)

    print("Green intensity:", green_intensity)

    reduction_rate = 1 - (scii13.dot(sol)) / (ci13_b)

    print("Reduction rate:", reduction_rate)
    optimal_pfs13[t] = sol

Optimal portfolio for t = 0 : [ 1.62643034e-01  2.21057457e-02  2.75714482e-01 -3.40804905e-20
  1.13366613e-01  2.27076307e-01  4.14801068e-03  1.92414053e-02
 -3.33005779e-20  1.75704402e-01 -1.83996096e-20]
Tracking error volatility: 0.11039174458544963
Carbon intensity: 186.907
Carbon momentum: -0.009731308447846278
Green intensity: 0.0029294828793553274
Reduction rate: 0.29999999999999993
Optimal portfolio for t = 1 : [ 1.75093711e-01  1.82292354e-02  2.36804167e-01 -7.11109023e-21
  1.30432079e-01  2.35711723e-01 -1.11513541e-20  2.22510070e-02
 -6.62418487e-21  1.81478079e-01 -3.50268986e-21]
Tracking error volatility: 0.1077651657462122
Carbon intensity: 173.82350999999997
Carbon momentum: -0.010106219053197365
Green intensity: 0.002858633647112127
Reduction rate: 0.3490000000000001
Optimal portfolio for t = 2 : [ 1.86581901e-01  1.41040979e-02  1.98440881e-01 -5.87580138e-21
  1.46167633e-01  2.43539728e-01 -9.24641267e-21  2.45837332e-02
 -5.48169593e-21  1.86582026e-01 -2.90

### Question 2.d

In [134]:
for key12, key13 in zip(optimal_pfs12, optimal_pfs13):
    implied_expected_return = expected_returns.dot(optimal_pfs12[key12])
    print(
        "Implied expected, taking into consideration 1 and 2, return for t =",
        key12,
        ":",
        np.sum(implied_expected_return - expected_returns),
    )
    implied_expected_return = expected_returns.dot(optimal_pfs13[key13])
    print(
        "Implied expected, taking into consideration 1, 2 and 3, return for t =",
        key13,
        ":",
        np.sum(implied_expected_return - expected_returns),
    )

Implied expected, taking into consideration 1 and 2, return for t = 0 : -0.1558792538222018
Implied expected, taking into consideration 1, 2 and 3, return for t = 0 : -0.10384958265614318
Implied expected, taking into consideration 1 and 2, return for t = 1 : -0.15562280646335774
Implied expected, taking into consideration 1, 2 and 3, return for t = 1 : -0.09261172673680355
Implied expected, taking into consideration 1 and 2, return for t = 2 : -0.1553843104196329
Implied expected, taking into consideration 1, 2 and 3, return for t = 2 : -0.08151920118153329
Implied expected, taking into consideration 1 and 2, return for t = 5 : -0.15476439790850877
Implied expected, taking into consideration 1, 2 and 3, return for t = 5 : -0.05268687648431916
Implied expected, taking into consideration 1 and 2, return for t = 10 : -0.1507053723546123
Implied expected, taking into consideration 1, 2 and 3, return for t = 10 : -0.008703963671839302


### Question 2.e
We expect the carbon intensity to be the same as at $t=0$ before the rebalancing, as the weights of the assets are the same when no rebalancing occurs.

### Question 3.a

In this question we have a constraint on weights where : 
$$ w_i \geq \frac{b_i}{2}$$

So in this case the QP program corresponds to: 

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = \frac{1}{2}b_n $ and $ w^+ = 1_n $ (with $b_n$ the vector of weights of the benchmark).
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint : $ C = \mathcal{CI}^\mathsf{T} $ and $ D = \mathcal{CI}^⋆ $

In [136]:
def solve_esg_qp_3_problem(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        weights/2 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci_b,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

Here we just considere the time $t=0$:

In [137]:
t = 0
sol = solve_esg_qp_3_problem(t)
print("Optimal portfolio for t =", t, ":", sol)
tracking_eror_volatility = np.sqrt((sol - weights).T.dot(cov_mat).dot(sol - weights))
print("Tracking error volatility:", tracking_eror_volatility)
carbon_momentum = cm12.dot(sol)
print("Carbon momentum:", carbon_momentum)
carbon_intensity = sci12.dot(sol)
print("Carbon intensity:", carbon_intensity)
green_intensity = gii.dot(sol)
print("Green intensity:", green_intensity)
reduction_rate = 1 - (sci12 * sol) / (ci_b)
print("Reduction rate:", reduction_rate)

Optimal portfolio for t = 0 : [0.041      0.0615     0.39598212 0.0155     0.066      0.12720186
 0.051      0.115      0.0225     0.08802126 0.01629475]
Tracking error volatility: 0.09560531390610101
Carbon momentum: -0.04320317787171007
Carbon intensity: 88.9476
Green intensity: 0.0052660565549487275
Reduction rate: [0.99225611 0.97386439 0.85353386 0.94705984 0.99013127 0.97897788
 0.95785721 0.97918437 0.90101757 0.93834882 0.78776869]


### Question 3.b

In this case we have another constraint to add : 
$$\mathcal{CM}(t,w):= \sum_{i=1}^{n} w_i \mathcal{CM}_i \leq \mathcal{CM}^*$$
with $\mathcal{CM}^* \in \{ −5\%, −6\%, −7\%, −8\% \}$

So the general formulation of the QP problem is:

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = 0 $ and $ w^+ = 1_n $.
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint and carbon momentum: C = $ \begin{pmatrix} \mathcal{CI}^\mathsf{T} \\ \mathcal{CM}^\mathsf{T} \end{pmatrix}$ and D = $ \begin{pmatrix} \mathcal{CI}^⋆ \\ \mathcal{CM}^⋆ \end{pmatrix}$.


In [98]:
cm_star_list = np.array([-5, -6, -7, -8])
cm_star_list = cm_star_list / 100 # convert from percentage to decimal

In [138]:
def solve_esg_qp_3_b_problem(t: float, i : int) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci_b,
        cp.sum(cm12 * w) <= cm_star_list[i],
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [151]:
for i in range(len(cm_star_list)):
    print("For CM* = ", cm_star_list[i])
    t = 0
    sol = solve_esg_qp_3_b_problem(t, i)
    print("Optimal portfolio for t =", t, ":", sol)
    tracking_eror_volatility = np.sqrt((sol - weights).T.dot(cov_mat).dot(sol - weights))
    print("Tracking error volatility:", tracking_eror_volatility)
    carbon_momentum = cm12.dot(sol)
    print("Carbon momentum:", carbon_momentum)
    carbon_intensity = sci12.dot(sol)
    print("Carbon intensity:", carbon_intensity)
    green_intensity = gii.dot(sol)
    print("Green intensity:", green_intensity)
    reduction_rate = 1 - (sci12 * sol) / (ci_b)
    print("Reduction rate:", reduction_rate)

For CM* =  -0.06
Optimal portfolio for t = 0 : [ 1.54577839e-03  6.62608788e-02  2.90188979e-01 -4.79718914e-22
  1.35337318e-01  2.74804856e-01  1.58397760e-01  1.94184481e-21
 -4.28020623e-22  4.78059857e-02  2.56584431e-02]
Tracking error volatility: 0.10422924645523664
Carbon momentum: -0.06
Carbon intensity: 88.9476
Green intensity: 0.007620052445919704
Reduction rate: [0.99970804 0.97184116 0.8926647  1.         0.97976352 0.95458414
 0.8691113  1.         1.         0.9665161  0.66581104]
For CM* =  0.0
Optimal portfolio for t = 0 : [ 9.52079102e-02  3.33688277e-02  4.46521686e-01 -7.42421262e-22
  2.18675153e-02  1.78335631e-01  6.53289731e-02  3.57770471e-21
 -6.66484141e-22  1.34675829e-01  2.46936267e-02]
Tracking error volatility: 0.13190031530001348
Carbon momentum: -0.04046482958791959
Carbon intensity: 88.9476
Green intensity: 0.006028154022616099
Reduction rate: [0.98201758 0.98581927 0.83484025 1.         0.99673023 0.97052721
 0.94601676 1.         1.         0.905671

### Question 3.c

In this case we have another constraint to add : 
$$\mathcal{GI}(t,w):= \sum_{i=1}^{n} w_i \mathcal{GI}_i \geq (1+\mathcal{G})\mathcal{GI}(b)$$
with $\mathcal{G} \geq 1$.

So the general formulation of the QP problem is:

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = 0 $ and $ w^+ = 1_n $.
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint and carbon momentum: C = $ \begin{pmatrix} \mathcal{CI}^\mathsf{T} \\ \mathcal{GI}^\mathsf{T} \end{pmatrix}$ and D = $ \begin{pmatrix} \mathcal{CI}^⋆ \\ -(1+\mathcal{G})\mathcal{GI}(b) \end{pmatrix}$.


In [101]:
G_list = np.array([0, 0.5, 1, 2])

In [147]:
def solve_esg_qp_3_c_problem(t: float, i : int) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci_b, 
        cp.sum(gii * w) <= (1+ G_list[i] )*gi_b
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [150]:
for i in range(len(G_list)):
    print("For G = ", G_list[i])
    t = 0
    sol = solve_esg_qp_3_c_problem(t, i)
    print("Optimal portfolio for t =", t, ":", sol)
    tracking_eror_volatility = np.sqrt((sol - weights).T.dot(cov_mat).dot(sol - weights))
    print("Tracking error volatility:", tracking_eror_volatility)
    carbon_momentum = cm12.dot(sol)
    print("Carbon momentum:", carbon_momentum)
    carbon_intensity = sci12.dot(sol)
    print("Carbon intensity:", carbon_intensity)
    green_intensity = gii.dot(sol)
    print("Green intensity:", green_intensity)
    reduction_rate = 1 - (sci12 * sol) / (ci_b)
    print("Reduction rate:", reduction_rate)

For G =  0.25
Optimal portfolio for t = 0 : [ 9.52079102e-02  3.33688277e-02  4.46521686e-01 -7.42421262e-22
  2.18675153e-02  1.78335631e-01  6.53289731e-02  3.57770471e-21
 -6.66484141e-22  1.34675829e-01  2.46936267e-02]
Tracking error volatility: 0.13190031530001348
Carbon momentum: -0.04046482958791959
Carbon intensity: 88.9476
Green intensity: 0.006028154022616099
Reduction rate: [0.98201758 0.98581927 0.83484025 1.         0.99673023 0.97052721
 0.94601676 1.         1.         0.90567138 0.67837731]
For G =  0.5
Optimal portfolio for t = 0 : [ 9.52079102e-02  3.33688277e-02  4.46521686e-01 -7.42421262e-22
  2.18675153e-02  1.78335631e-01  6.53289731e-02  3.57770471e-21
 -6.66484141e-22  1.34675829e-01  2.46936267e-02]
Tracking error volatility: 0.13190031530001348
Carbon momentum: -0.04046482958791959
Carbon intensity: 88.9476
Green intensity: 0.006028154022616099
Reduction rate: [0.98201758 0.98581927 0.83484025 1.         0.99673023 0.97052721
 0.94601676 1.         1.       

### Question 3.c

In this case we have all the constraints to take into account.

So the general formulation of the QP problem is:

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = \frac{1}{2} b_n $ and $ w^+ = 1_n $.
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint and carbon momentum: C = $ \begin{pmatrix} \mathcal{CI}^\mathsf{T} \\ \mathcal{GI}^\mathsf{T} \end{pmatrix}$ and D = $ \begin{pmatrix} \mathcal{CI}^⋆ \\ -(1+\mathcal{G})\mathcal{GI}(b) \end{pmatrix}$.

In [149]:
cm_star_list = np.array([-6, 0, -7, -7])
cm_star_list = cm_star_list / 100 # convert from percentage to decimal

G_list = np.array([0.25, 0.5, 0, 25])

def solve_esg_qp_3_d_problem(t: float, i : int , j: int) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci_b, 
        cp.sum(cm12 * w) <= cm_star_list[i],
        cp.sum(gii * w) <= (1+ G_list[j] )*gi_b
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [152]:
for i in range(len(G_list)):
    print("For CM* = ", cm_star_list[i], " and G = ", G_list[i])
    t = 0
    sol = solve_esg_qp_3_d_problem(t, i,i)
    print("Optimal portfolio for t =", t, ":", sol)
    tracking_eror_volatility = np.sqrt((sol - weights).T.dot(cov_mat).dot(sol - weights))
    print("Tracking error volatility:", tracking_eror_volatility)
    carbon_momentum = cm12.dot(sol)
    print("Carbon momentum:", carbon_momentum)
    carbon_intensity = sci12.dot(sol)
    print("Carbon intensity:", carbon_intensity)
    green_intensity = gii.dot(sol)
    print("Green intensity:", green_intensity)
    reduction_rate = 1 - (sci12 * sol) / (ci_b)
    print("Reduction rate:", reduction_rate)

For CM* =  -0.06  and G =  0.25
Optimal portfolio for t = 0 : [ 1.54577839e-03  6.62608788e-02  2.90188979e-01 -4.79718914e-22
  1.35337318e-01  2.74804856e-01  1.58397760e-01  1.94184481e-21
 -4.28020623e-22  4.78059857e-02  2.56584431e-02]
Tracking error volatility: 0.10422924645523664
Carbon momentum: -0.06
Carbon intensity: 88.9476
Green intensity: 0.007620052445919704
Reduction rate: [0.99970804 0.97184116 0.8926647  1.         0.97976352 0.95458414
 0.8691113  1.         1.         0.9665161  0.66581104]
For CM* =  0.0  and G =  0.5
Optimal portfolio for t = 0 : [ 9.52079102e-02  3.33688277e-02  4.46521686e-01 -7.42421262e-22
  2.18675153e-02  1.78335631e-01  6.53289731e-02  3.57770471e-21
 -6.66484141e-22  1.34675829e-01  2.46936267e-02]
Tracking error volatility: 0.13190031530001348
Carbon momentum: -0.04046482958791959
Carbon intensity: 88.9476
Green intensity: 0.006028154022616099
Reduction rate: [0.98201758 0.98581927 0.83484025 1.         0.99673023 0.97052721
 0.94601676 1